# Treating hotspot detection as a classification problem

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# read data
df = pd.read_csv(r'./data/NY_Taxi_Rich.csv')

In [51]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
from sklearn.cluster import KMeans


def plot_on_ny_map(df, points=10, figsize=(12,12), save=False, title='NYC data', fname='plot.png', cols=['startLongitude', 'startLatitude'], colored=False, color_col=None, size=12):
    data_x = df[cols[0]][:points]
    data_y = df[cols[1]][:points]

    ny = gpd.read_file(gpd.datasets.get_path('nybb'))
    geometry = [Point(x, y) for x, y in zip(data_x, data_y)]
    crs = 'EPSG:4326'
    
    gdf = GeoDataFrame(df[[cols[0], cols[1]]][:points], crs=crs, geometry=geometry)
    
    fig, ax = plt.subplots(figsize = (12,12))
    ax.set_title(title)
    ny.to_crs(epsg=4326).plot(ax=ax, color='lightgrey', edgecolor='black')
    
    if colored:
        if color_col is None:
            raise Exception("color_col arg is missing, but colored is set to True.")
            return
        gdf.plot(ax=ax, markersize=size, c=df[color_col])
    else:
        gdf.plot(ax=ax, markersize=size)
    
    if save:
        plt.savefig(fname)
    else:
        plt.show()

def clean_spatial_data(df, cols=['startLatitude', 'startLongitude'], stddevs=3):
    for col in cols:
        std = df[col].std()
        mean = df[col].mean()
        lower_boundary = mean - stddevs * std
        upper_boundary = mean + stddevs * std
        df = df.loc[(df[col] > lower_boundary) & (df[col] < upper_boundary)]
    return df

def encode_text_fields(df, cols=['season', 'dayName', 'dayPeriod']):
    for col in cols:
        unique_vals = df[col].unique()
        for i in range(len(unique_vals)):
            df.loc[df[col] == unique_vals[i], col] = i
    return df

def label_data(df, n = 120):
    X = df[['startLatitude', 'startLongitude']]
    kmeans = KMeans(n_clusters = n).fit(X)
    labels = pd.DataFrame(kmeans.labels_, columns=['label'])
    labels = labels.set_index(df.index)
    result = df.join(labels)
    return result

def get_training_and_test_data(labeled_df, feature_cols=['year', 'month', 'day', 'hour', 'minute', 'second', 'season', 'dayName', 'dayPeriod', 'temperature', 'rain', 'snow'], label_col='label', split = 0.8, random = True):
    rows, _ = labeled_df.shape
    training_rows = int(rows * 0.8)
    
    if random:
        labeled_df = labeled_df.sample(frac = 1.0)
    
    training_data = labeled_df[:training_rows]
    test_data = labeled_df[training_rows:rows]
    
    return training_data[feature_cols], training_data['label'], test_data[feature_cols], test_data['label']

def convert_labels_to_vectors(labels, shape = (120,)):
    collector = []
    for l in labels:
        label_vector = np.zeros(shape=shape)
        label_vector[l] = 1.0
        collector.append(label_vector)
    df = pd.DataFrame(collector)
    return df

def max_abs_scaler(df, cols=['year', 'month', 'day', 'hour', 'minute', 'second', 'season', 'dayName', 'dayPeriod', 'temperature', 'rain', 'snow']):
    for col in cols:
        df[col] = df[col].abs()
        df[col] = df[col] / df[col].max()
    return df

Basic model idea: The columns **year**, **month**, **day**, **hour**, **minute**, **second**, **season (encoded)**, **dayName (encoded)**, **dayPeriod (encoded)**, **temperature**, **rain** and **snow**, determine the class i. e. the location of the trip. Since the classifier will try to predict a probability for each class at the same time it outputs the probability for any location directly. A probability abova a certain threshold can be considered a hotspot. 

Further Data insights below, show there are no invalid values in the relevant columns. There is therfore no need to preprocess the data any further at the moment.

In [4]:
print('Year: \t', df['year'].unique())
value = df['month'].unique()
value.sort()
print('Month: \t', value)
value = df['day'].unique()
value.sort()
print('Day: \t', value)
value = df['hour'].unique()
value.sort()
print('Hour: \t', value)
print('Rain: \t', df['rain'].unique())
print('Snow: \t', df['snow'].unique())

Year: 	 [2016]
Month: 	 [1 2 3 4 5 6]
Day: 	 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
Hour: 	 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Rain: 	 [1 0]
Snow: 	 [0 1]


In [5]:
# get data for a slice of df

df3 = df[:20000]
df3.head()
df3 = label_data(df3)
df3 = encode_text_fields(df3)
X_tr, y_tr, X_test, y_test = get_training_and_test_data(df3)

In [6]:
# set up SVM as classfier

from sklearn import svm

clf = svm.SVC(kernel='poly')
clf.decision_function_shape = "ovr"
clf.probability = True

clf.fit(X_tr, y_tr)
predictions = clf.predict(X_test)
probabilities = clf.predict_proba(X_test)

In [7]:
probabilities[0]

array([0.00759351, 0.01939613, 0.00852676, 0.01074235, 0.01104069,
       0.00369098, 0.01585716, 0.01315833, 0.00107017, 0.02634778,
       0.01460706, 0.00202821, 0.00486618, 0.00872068, 0.00225122,
       0.01505428, 0.01207196, 0.01656893, 0.01232817, 0.02304663,
       0.01246093, 0.00040374, 0.00023088, 0.01020624, 0.01473615,
       0.00148761, 0.0087028 , 0.01231541, 0.00221089, 0.00119856,
       0.00046724, 0.001327  , 0.01256582, 0.01914482, 0.00159776,
       0.01375627, 0.01228814, 0.01320093, 0.00168494, 0.00927564,
       0.0006076 , 0.00011573, 0.01125696, 0.00961119, 0.00943441,
       0.00126004, 0.01044554, 0.00437813, 0.01033246, 0.01087538,
       0.01291434, 0.00013412, 0.01229846, 0.01810545, 0.0009958 ,
       0.0226185 , 0.00296616, 0.01933703, 0.02309613, 0.00055392,
       0.01070386, 0.01029328, 0.00047874, 0.01112175, 0.00077048,
       0.01971891, 0.01034809, 0.01509344, 0.01628585, 0.00569401,
       0.00284521, 0.01220359, 0.01146758, 0.01387692, 0.01979

In [55]:
from keras.models import Sequential
from keras.layers import Dense


df3 = df[:20000]
df3.head()
df3 = label_data(df3)
df3 = encode_text_fields(df3)
X_tr, y_tr, X_test, y_test = get_training_and_test_data(df3)

for col in X_tr.columns:
    X_tr[col] = X_tr[col].astype(float)
    X_test[col] = X_test[col].astype(float)

model = Sequential()
model.add(Dense(20, input_dim=12, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(120, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

tr_vectors = convert_labels_to_vectors(y_tr)
test_vectors = convert_labels_to_vectors(y_test)
model.fit(X_tr, tr_vectors, epochs=100)

Epoch 1/100
500/500 [==============================] - 5s 5ms/step - loss: 7.2787
Epoch 2/100
500/500 [==============================] - 2s 5ms/step - loss: 0.1184
Epoch 3/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0970
Epoch 4/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0889
Epoch 5/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0871
Epoch 6/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0820
Epoch 7/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0869
Epoch 8/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0825
Epoch 9/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0847
Epoch 10/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0810
Epoch 11/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0766
Epoch 12/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0807
Epoch 13/100


500/500 [==============================] - 2s 4ms/step - loss: 0.0450
Epoch 100/100
500/500 [==============================] - 2s 4ms/step - loss: 0.0450


In [64]:
from sklearn.metrics import accuracy_score
yhat = model.predict(X_test)

In [111]:
import math

for i in range(1, yhat.shape[0]):
    s = np.subtract(yhat[i], (np.array(test_vectors[:i])[0]))
    err = 0
    for j in s:
        err += math.pow(j, 2)
    print(err / 120)

0.008170263333952328
0.008170252484265566
0.008169481730276233
0.008171552393415906
0.008170513430596401
0.008169366053970568
0.008170602238889931
0.00816975624458428
0.008170430770760862
0.008170857991704254
0.008171173189694767
0.008171036025564826
0.008170006954357264
0.008170515913598329
0.008170749476873016
0.008171089207846214
0.008170907409887962
0.008170854642427517
0.008171072523881821
0.008170344819731792
0.00817093098891765
0.008169675770452292
0.008170129399303012
0.008170986319362309
0.00817108078259118
0.008171338350830665
0.008170288334555598
0.008170228208159961
0.008170870637967542
0.008173715657073264
0.008169825137939724
0.008170871172090548
0.00817296580866372
0.008170761080727264
0.00817106959385288
0.008171102388127181
0.008170695374575996
0.008170752003834028
0.008170182265076263
0.008169583172176814
0.008170886072822429
0.008170594017620986
0.00817137157057831
0.008170883926308138
0.008170913275281534
0.008171209306087092
0.008170666567521257
0.00816945923491328

0.008169852221842322
0.008170179164720728
0.008171257273719574
0.008170895348562517
0.008170330688278408
0.008171178985080681
0.008170388774204382
0.008169650179739066
0.008170022766740767
0.00817121181763819
0.008170758619163603
0.008170145230425696
0.008169888892025424
0.008171006958171115
0.008170094110590912
0.008169665933248318
0.008170097457407592
0.008170787778557887
0.008170995072316894
0.008170734615691701
0.008170507124679472
0.008170654233937334
0.008170998190008242
0.008170773950961548
0.008171208683940057
0.008171062245902451
0.008171104274640125
0.008170376448466759
0.008171097338701656
0.008170551067384731
0.008170775889594322
0.008171605433618761
0.00817097778548924
0.008170654995156339
0.008170329714141744
0.008169545260093324
0.008169246920804518
0.008170893237198457
0.0081711983344312
0.00817110107565907
0.00817130708354258
0.008170407796616593
0.008170536315821748
0.008170988153841064
0.008171328996677227
0.008169935947127387
0.008170057529238803
0.00817017126506590

0.008170115243842974
0.00817061509119088
0.008169799757164059
0.008170932900243053
0.008171156048992495
0.008170320934587142
0.0081701846397162
0.008170844479804908
0.008170477234764056
0.008170093528212801
0.008169865399015867
0.008169912637954916
0.008170496494533013
0.008171184447749485
0.008170425643346965
0.008171081626024975
0.008170597268609596
0.008170792025089224
0.008170706511799393
0.00817071294451702
0.008170688650458464
0.008171225856225107
0.008169836587363219
0.008170367595668237
0.00817113221716215
0.008171235177870677
0.00816987402725112
0.008170871728927113
0.008171048836765697
0.00817087926851186
0.00816943718425107
0.00817056479936689
0.008170525189975679
0.008170731205155308
0.00817040158608824
0.008170719534300664
0.008170036320198503
0.008171924378814461
0.008171012499791368
0.008170687704136549
0.00817011361216158
0.008170591347708844
0.008170291102264884
0.008170964359356698
0.008170898428354204
0.008170387513930252
0.008170157565179356
0.008170268721654854
0.0

0.00817085744166847
0.008170633548812584
0.008170601857320985
0.008169926863458453
0.008171306500278395
0.008170933523950617
0.00817088897771789
0.008170853911799468
0.008170952487800436
0.008170046225261579
0.008169789788686842
0.008170911803872754
0.008171194623004913
0.008171089825861602
0.008169862676257607
0.008171415711260023
0.008169768873989432
0.008169988676702508
0.008169658423647188
0.008171095940429859
0.008169209505164775
0.008170975389850964
0.008170835861646965
0.008170726811304653
0.008171330483833861
0.008170353624601574
0.008170770499878958
0.008170717067457336
0.008170537235260808
0.008170953620913505
0.008169718129183592
0.00817095355890822
0.008170736584113845
0.00817094385265648
0.008170769126496336
0.00817114393619116
0.00817111753306333
0.008170566174386
0.00817030523468454
0.008170401992377278
0.008171122121389892
0.008171193229756851
0.00817088630000948
0.008169871949290775
0.0081691864160461
0.008170378794938746
0.008169646557549332
0.008169592457955165
0.008

0.008170878985133826
0.008169639140614181
0.008169714408598752
0.008170627640271298
0.008170120366085817
0.008170886532313123
0.008170621956832536
0.008170024979818473
0.008170758324026198
0.008170689903261662
0.008171165368381712
0.008170868985823494
0.008169955940038335
0.00816981302223132
0.008170703365242452
0.008170559813713448
0.008170373791974953
0.008170111964166466
0.008170903900816491
0.008170210888678418
0.008170428039390961
0.008170351187754641
0.008170160960985216
0.00817019281775839
0.008169992797165689
0.008169932722216918
0.008171014782835077
0.008170941475062584
0.008169802903689597
0.008170815461398678
0.008171082106932093
0.008171366936003791
0.008169971386152698
0.008169976760608539
0.008171249110387021
0.008171043894448627
0.008169689867814751
0.008170025929194276
0.00817089503822835
0.008170942894775771
0.008171379643197852
0.008171215742098163
0.008169738079937025
0.008170459396024258
0.008170376752675434
0.008170913489271786
0.008170641473252851
0.00817120442598

0.008169866773747614
0.008170902299367637
0.00817059297941868
0.008171022322842182
0.00816958368201466
0.008170239170971183
0.008170194402654303
0.008171179382196261
0.00817001195655823
0.008170934760110962
0.008170831775241211
0.008171015096649083
0.008170715583652345
0.008170548926164973
0.008170454085469726
0.008170480346564481
0.008170602405081195
0.008170160541438093
0.008170915969467674
0.008169898538922053
0.008171011335006674
0.008171070762252652
0.008171029351113452
0.008170750339627515
0.00817083523076794
0.008171434857165118
0.008170812558729644
0.008171289137226622
0.008170894577720338
0.008170938343659772
0.008170862306247091
0.008170749243374143
0.008170393372050873
0.008170864805510155
0.0081700792246808
0.008169734888133889
0.008171337327087816
0.008170026805305574
0.00817002346750598
0.008170585849954075
0.008170694017872819
0.00817073366870673
0.008171102780916343
0.008170825954072567
0.008170616255492747
0.008171040680018461
0.008170428368368931
0.008169724645932143


0.008171263865487104
0.008170119099839033
0.008171015760925117
0.008170143967322916
0.008170441337382556
0.008169641549470528
0.00817092419966389
0.00817046470283632
0.008171287317463612
0.008171284061094386
0.008171031128666259
0.008171310608815867
0.008171113794627687
0.008170798394968102
0.008170638115169737
0.008171640180362295
0.008170856229105477
0.008170397683026114
0.008170052536027962
0.008170729009722853
0.008170005912609085
0.008170364885964686
0.00817052464295953
0.008170334519036081
0.00817008128528645
0.008170946231904357
0.008170975997665501
0.008170926506175177
0.0081697474430579
0.008171028114272684
0.008170806323974746
0.008170639116392469
0.008170488867774092
0.008171016754049241
0.008170759449089354
0.00817062388025126
0.008170412876827411
0.008171285401502066
0.008170968409296226
0.008170432076512978
0.008170174570996289
0.008171118967947671
0.008170109255067054
0.008170742282481418
0.008169808782956376
0.00817080432635458
0.008170668800805348
0.008170107620265874


0.008170311459238798
0.008170303843951355
0.008171253432817676
0.008171004952931122
0.008169632938058571
0.008169500550831457
0.008170754704170616
0.008170226575448102
0.008171403737865415
0.008171206809450353
0.008169767418386984
0.008169765537167164
0.008170371864472491
0.008170207860167254
0.00817110105768273
0.00817107542700019
0.008171040779524436
0.008170869637863891
0.0081706776804736
0.00816947073021424
0.008170578866309028
0.008171638659196288
0.008171130213276204
0.008171053620945957
0.008170758972071512
0.008169555416015307
0.008169528327436493
0.008170335471201365
0.008170125507755303
0.00816996918416971
0.008170930194846847
0.00817040934503795
0.008170668133994476
0.00817098008958303
0.00817076456207968
0.00817115671191745
0.00816977077740685
0.008171290791419433
0.008170161481833176
0.008170905916771614
0.008171102933578934
0.00817113596391846
0.0081701603056186
0.00817119402647613
0.008170486576582462
0.008171057595876375
0.008171073665802274
0.00817101954548952
0.008171

0.008170942322785034
0.008169954173387265
0.00817093911586809
0.008170510015931198
0.008170530556219245
0.008170140843763957
0.008171394108819608
0.008170016422108435
0.008170209723220492
0.008170211108453538
0.008169871008922499
0.008170092175022663
0.00817131512608364
0.008170206608599054
0.008170453504356873
0.008169540796023465
0.00817066355104776
0.008170688506613451
0.00817128551999485
0.008171035578510856
0.008170212014994906
0.00816950954506444
0.008170864093219729
0.00817086446252483
0.008170586922724684
0.008171397719714441
0.00817080573748786
0.008170080337092666
0.00817048293387738
0.008170605953712868
0.008171239985796917
0.008170438395600294
0.008170577443132475
0.008170939438028432
0.008170751334441334
0.00817065525186026
0.008171381761043527
0.008170786569592765
0.008170436906163184
0.008170536529915132
0.008170477933258317
0.008170430173050785
0.008170732374321204
0.00817132070902735
0.008170736603762342
0.008169560607175034
0.008170631839049928
0.008170169608194612
0.